# Optimizer Example

This notebook demonstrates how to use the PyQuOpt numerical optimization code to generate novel implementations of a specified unitary matrix. In this case, the Toffoli gate is explored.

### Imports

In [1]:
import sys
import numpy as np

sys.path.append('..')
from pyquopt import *

### Optimization Hyperparameters

In [93]:
num_qubits = 3
num_mq_instructions = 8  # want to explore circuits with six multi-qubit gates
num_params = 3 * num_qubits * (num_mq_instructions + 1)  # formula explained in MICRO paper

mq_dict = {
    0: ThreeGates.CX01,
    1: ThreeGates.CX02,
}

In [94]:
alpha = 0.30  # penalty for non-standard angles
gamma = 0   # penalty for large angles
non_fixed_params = np.ones(num_params)#np.array([0,0,0,0,0,0,0,0,0] + [1] * (num_params-9))   # leave all parameters unfixed
fixed_params_vals = np.zeros(num_params)  # unfixed parameters should have a "fixed" value of 0

In [95]:
mq_instructions = [0, 0, 1, 0, 1, 0, 1, 0]

### Instantiate and Run Optimizer

In [96]:
optimizer = Optimizer(num_qubits=num_qubits, mq_instructions=mq_instructions, mq_dict=mq_dict, target=ThreeGates.TOFFOLI,
                     alpha=alpha, gamma=gamma, non_fixed_params=non_fixed_params, fixed_params_vals=fixed_params_vals)

In [97]:
opt_params, opt_val = optimizer.find_parameters_least_squares(40)

Note that there is also a function `optimizer.find_parameters_least_squares_par(num_guesses, num_procs)` that allows for process-based parallelism.

In [98]:
opt_val

0.6139130752839813

In [99]:
opt_params_rounded = round_params(opt_params)
print(opt_params_rounded)

[100. 265.  90. 330.  80. 275. 265. 300. 225.   0. 135. 135.   0. 315.
  45. 315. 350.  90.  60. 175. 210. 180.  95.  80. 150.  45. 265. 300.
  30. 300. 100. 210. 330. 240. 225.  30. 315. 120. 150. 315.  45. 175.
 300. 330.  45. 315.  30. 275.  30. 135. 355. 150. 330. 260. 300. 135.
 300. 100.  60. 240. 315. 300. 135.  45. 300. 330. 225. 150. 315. 275.
 150. 180.   0. 150. 330. 270. 180. 260. 240. 240. 350.]


### Verify Implementation Accuracy

In [100]:
ub = UnitaryBuilder(num_qubits, mq_instructions, mq_dict)

In [101]:
implementation_matrix = ub.build_unitary(opt_params_rounded * np.pi / 180)

In [102]:
print(get_unitary_infidelity(ThreeGates.TOFFOLI, implementation_matrix, 8))

0.15242410865555056


[0, 0, 1, 0, 1, 0, 1, 0] 0.15883888520024747 

[0, 1, 1, 0, 1, 0, 1, 0] 0.16106198458460275

[0, 1, 0, 1, 1, 0, 1, 0] 0.16007174700062432

[1, 0, 1, 0, 1, 0, 1, 1] 0.1620227861829251
[0, 0, 1, 1, 0, 1, 0, 1] 0.15371601818536684

An infidelity less than 0.01 is acceptable.

In [103]:
params_list = []
for i in range(0, 64):
    temp = "{0:06b}".format(i)
    temp = [0] + [int(d) for d in temp] + [0]
    print(temp)
    params_list.append(temp)

[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 0]
[0, 0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 1, 0, 1, 1, 0]
[0, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 1, 0]
[0, 0, 0, 1, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 1, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1, 0]
[0, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 1, 0]
[0, 0, 1, 0, 1, 1, 0, 0]
[0, 0, 1, 0, 1, 1, 1, 0]
[0, 0, 1, 1, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 1, 0]
[0, 0, 1, 1, 0, 1, 0, 0]
[0, 0, 1, 1, 0, 1, 1, 0]
[0, 0, 1, 1, 1, 0, 0, 0]
[0, 0, 1, 1, 1, 0, 1, 0]
[0, 0, 1, 1, 1, 1, 0, 0]
[0, 0, 1, 1, 1, 1, 1, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 1, 1, 0]
[0, 1, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 1, 0]
[0, 1, 0, 0, 1, 1, 0, 0]
[0, 1, 0, 0, 1, 1, 1, 0]


In [ ]:
useful_params = []
useful_bitstring = []
for param in params_list[::-1]:
    mq_instructions = param
    print(mq_instructions)
    optimizer = Optimizer(num_qubits=num_qubits, mq_instructions=mq_instructions, mq_dict=mq_dict, target=ThreeGates.TOFFOLI,
                     alpha=alpha, gamma=gamma, non_fixed_params=non_fixed_params, fixed_params_vals=fixed_params_vals)
    opt_params, opt_val = optimizer.find_parameters_least_squares(20)
    opt_params_rounded = round_params(opt_params)
    implementation_matrix = ub.build_unitary(opt_params)
    infidelity = get_unitary_infidelity(ThreeGates.TOFFOLI, implementation_matrix, 8)
    print(infidelity)
    if infidelity < 0.1:
        useful_params.append(opt_params_rounded)
        useful_bitstring.append(non_fixed_params)

[0, 1, 1, 1, 1, 1, 1, 0]
0.9996119211826848
[0, 1, 1, 1, 1, 1, 0, 0]
0.9985722245700315
[0, 1, 1, 1, 1, 0, 1, 0]
0.6942980667984275
[0, 1, 1, 1, 1, 0, 0, 0]
0.956682026232788
[0, 1, 1, 1, 0, 1, 1, 0]
0.9644323152389117
[0, 1, 1, 1, 0, 1, 0, 0]
0.9849192137861458
[0, 1, 1, 1, 0, 0, 1, 0]
0.9509007298751495
[0, 1, 1, 1, 0, 0, 0, 0]
0.9903291247175348
[0, 1, 1, 0, 1, 1, 1, 0]
0.9745276098515491
[0, 1, 1, 0, 1, 1, 0, 0]
0.9753126646279351
[0, 1, 1, 0, 1, 0, 1, 0]
0.7867310078346338
[0, 1, 1, 0, 1, 0, 0, 0]
0.9742428970441697
[0, 1, 1, 0, 0, 1, 1, 0]
0.947587637889504
[0, 1, 1, 0, 0, 1, 0, 0]
0.9918987500479347
[0, 1, 1, 0, 0, 0, 1, 0]
0.9757427206249581
[0, 1, 1, 0, 0, 0, 0, 0]
0.9989889631926911
[0, 1, 0, 1, 1, 1, 1, 0]
0.9952328491861211
[0, 1, 0, 1, 1, 1, 0, 0]
0.9616781306284383
[0, 1, 0, 1, 1, 0, 1, 0]
0.9972865453876614
[0, 1, 0, 1, 1, 0, 0, 0]
0.9804206315924563
[0, 1, 0, 1, 0, 1, 1, 0]
0.9678499167686035
[0, 1, 0, 1, 0, 1, 0, 0]
0.9974812592805555
[0, 1, 0, 1, 0, 0, 1, 0]
0.9924643